In [2]:
import pandas as pd
import numpy as np

from IPython.display import display

In [3]:
train = pd.read_csv('data/training.csv')

print(train.columns)
print("Dataset shape: ", train.shape)

Index(['left_eye_center_x', 'left_eye_center_y', 'right_eye_center_x',
       'right_eye_center_y', 'left_eye_inner_corner_x',
       'left_eye_inner_corner_y', 'left_eye_outer_corner_x',
       'left_eye_outer_corner_y', 'right_eye_inner_corner_x',
       'right_eye_inner_corner_y', 'right_eye_outer_corner_x',
       'right_eye_outer_corner_y', 'left_eyebrow_inner_end_x',
       'left_eyebrow_inner_end_y', 'left_eyebrow_outer_end_x',
       'left_eyebrow_outer_end_y', 'right_eyebrow_inner_end_x',
       'right_eyebrow_inner_end_y', 'right_eyebrow_outer_end_x',
       'right_eyebrow_outer_end_y', 'nose_tip_x', 'nose_tip_y',
       'mouth_left_corner_x', 'mouth_left_corner_y', 'mouth_right_corner_x',
       'mouth_right_corner_y', 'mouth_center_top_lip_x',
       'mouth_center_top_lip_y', 'mouth_center_bottom_lip_x',
       'mouth_center_bottom_lip_y', 'Image'],
      dtype='object')
Dataset shape:  (7049, 31)


In [4]:
## Dataset after dropping na

train_without_na = train.dropna()

print("Shape after dropping na: ", train_without_na.shape)

Shape after dropping na:  (2140, 31)


In [5]:
test = pd.read_csv('data/test.csv')
print(test.columns)
display(test.head())
print("Total test images: ", test.shape[0])
print("Images x dimensions : ", test.shape[0]*30)

Index(['ImageId', 'Image'], dtype='object')


,ImageId,Image
0,1,182 183 182 182 180 180 176 169 156 137 124 10...
1,2,76 87 81 72 65 59 64 76 69 42 31 38 49 58 58 4...
2,3,177 176 174 170 169 169 168 166 166 166 161 14...
3,4,176 174 174 175 174 174 176 176 175 171 165 15...
4,5,50 47 44 101 144 149 120 58 48 42 35 35 37 39 ...


Total test images:  1783
Images x dimensions :  53490


In [6]:
def to_img_arr(img_vals):
    img_arr = np.fromstring(img_vals, sep=' ')
    img_arr = img_arr.reshape((96,96))
    return img_arr

train_without_na['Image'] = train_without_na['Image'].apply(to_img_arr)
print("Image dimension: ", train_without_na['Image'][0].shape)

Image dimension:  (96, 96)


/home/saurabh/anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
print("Image dimension before reshape: ", np.fromstring(train['Image'][0], sep=' ').shape)

Image dimension before reshape:  (9216,)


In [8]:
print(max(train['left_eye_center_x']))
print(max(train['mouth_center_bottom_lip_x']))


94.68928
89.438592


In [9]:
img_vals = []
for img_val in train_without_na['Image']:
    img_vals.append(img_val)
    
#X = np.array(train_without_na['Image'].as_matrix())
X = np.array(img_vals)
print("X shape before expanding dims: ", X.shape)

X = np.expand_dims(X, axis=3)
print("X shape after expanding dims: ", X.shape)

X shape before expanding dims:  (2140, 96, 96)
X shape after expanding dims:  (2140, 96, 96, 1)


from keras.models import load_model

model = load_model('model.h5')
y = model.predict(np.expand_dims(X[0], axis=0), batch_size=1)
y_ = (y+0.5)*95
print(y)
print(y_)


y_all = model.predict(X)
print("Final shape: ", y_all.shape)
y_ = (y+0.5)*95

print(y_[:5])

In [10]:
idlookup = pd.read_csv('data/IdLookupTable.csv')
display(idlookup.head())

,RowId,ImageId,FeatureName,Location
0,1,1,left_eye_center_x,NaN
1,2,1,left_eye_center_y,NaN
2,3,1,right_eye_center_x,NaN
3,4,1,right_eye_center_y,NaN
4,5,1,left_eye_inner_corner_x,NaN


In [16]:
print(idlookup['ImageId'].max())
print(idlookup.shape)
print(len(idlookup['ImageId'].unique()))

1783
(27124, 4)
1783


In [20]:
idlookup_grpby = idlookup.groupby('ImageId').count().reset_index()
display(idlookup_grpby.head())
print(idlookup_grpby['FeatureName'].min())
display(idlookup_grpby[idlookup_grpby['FeatureName'] < 30])

,ImageId,RowId,FeatureName,Location
0,1,30,30,0
1,2,30,30,0
2,3,30,30,0
3,4,30,30,0
4,5,30,30,0


6


,ImageId,RowId,FeatureName,Location
61,62,28,28,0
404,405,28,28,0
411,412,24,24,0
413,414,24,24,0
414,415,28,28,0
415,416,26,26,0
420,421,28,28,0
423,424,28,28,0
426,427,28,28,0
429,430,26,26,0


In [21]:
test_output = pd.read_csv('data/test_output.csv')
display(test_output.head())

,RowId,ImageId,FeatureName,Location
0,1,1,left_eye_center_x,65.255875
1,2,1,left_eye_center_y,37.283890
2,3,1,right_eye_center_x,29.840532
3,4,1,right_eye_center_y,37.468510
4,5,1,left_eye_inner_corner_x,59.302391


In [25]:
idlookup_ = idlookup[['RowId', 'ImageId', 'FeatureName']]
df_final = test_output.merge(idlookup_, on=['ImageId', 'FeatureName'])
display(df_final)
print("Max rowId x: ", max(df_final['RowId_x']), max(df_final['RowId_y']))

,RowId_x,ImageId,FeatureName,Location,RowId_y
0,1,1,left_eye_center_x,65.255875,1
1,2,1,left_eye_center_y,37.283890,2
2,3,1,right_eye_center_x,29.840532,3
3,4,1,right_eye_center_y,37.468510,4
4,5,1,left_eye_inner_corner_x,59.302391,5
5,6,1,left_eye_inner_corner_y,38.195705,6
6,7,1,left_eye_outer_corner_x,71.668343,7
7,8,1,left_eye_outer_corner_y,38.276001,8
8,9,1,right_eye_inner_corner_x,36.565590,9
9,10,1,right_eye_inner_corner_y,37.655193,10


Max rowId x:  53490 27124
